In [1]:
import os
import cv2
import dlib
import numpy as np

In [4]:
# Load the face detector and face recognition model
detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor("shape_predictor_5_face_landmarks.dat")
facerec = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")

In [5]:
# Function to extract face descriptors from an image
def extract_face_descriptor(image_path):
    # Load the image
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Detect faces in the image
    faces = detector(image_rgb, 1)

    # If no faces are found, return None
    if len(faces) == 0:
        return None

    # Extract the first face found
    face = faces[0]
    
    # Get the landmarks and face descriptor
    shape = sp(image_rgb, face)
    face_descriptor = facerec.compute_face_descriptor(image_rgb, shape)

    return np.array(face_descriptor)

In [6]:
# Define the path to your dataset
dataset_path = "E:\\Datasets\\data_tango"

In [7]:
# Create a dictionary to store face embeddings in memory
face_embeddings_db = {}

# Iterate through each folder in the dataset
for label in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, label)
    if os.path.isdir(folder_path):
        for filename in os.listdir(folder_path):
            if filename.endswith(".jpg") or filename.endswith(".png"):
                image_path = os.path.join(folder_path, filename)
                descriptor = extract_face_descriptor(image_path)
                if descriptor is not None:
                    face_embeddings_db[label] = descriptor 

In [13]:
# Testing with a new face
input_image_path = r"C:\Users\DELL\Downloads\chandler.jpg" 
input_descriptor = extract_face_descriptor(input_image_path)

In [14]:
if input_descriptor is not None:
    
    similarities = {}
    for name, db_descriptor in face_embeddings_db.items():
        # Calculate the Euclidean distance between descriptors
        distance = np.linalg.norm(input_descriptor - db_descriptor)
        similarities[name] = distance

    # Sorting and results
    sorted_similarities = sorted(similarities.items(), key=lambda x: x[1])
    for name, dist in sorted_similarities[:5]:
        print(f'Name: {name}, Distance: {dist:.2f}')
else:
    print("No face detected in the input image.")

Name: 667, Distance: 0.46
Name: 370, Distance: 0.59
Name: 471, Distance: 0.60
Name: 122, Distance: 0.61
Name: 528, Distance: 0.62
